In [ ]:
!pip install --upgrade boto3 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install --upgrade sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
# For notebook instances (Amazon Linux)
!sudo yum update -y
!sudo yum install amazon-linux-extras
!sudo amazon-linux-extras install epel -y
!sudo yum update -y
!sudo yum install git-lfs git -y

In [ ]:
#下载模型snapshot到本地，需要25G空间
#需大约15-30分钟时间，请耐心等待, 如果左侧大括号内还是[*]，就还在下载中，*变成任意数例如[3]就证明已完成

from pathlib import Path
local_model_path = Path("./Baichuan2-13B-Chat-4bits")
local_model_path.mkdir(exist_ok=True)
model_name = "Baichuan-inc/Baichuan2-13B-Chat-4bits"
clone_path = f"https://www.wisemodel.cn/{model_name}.git"
print(clone_path)

!git lfs install
!git clone $clone_path

In [ ]:
#定义本地路径model_snapshot_path，以及s3路径s3_location，方便下一步从本地将模型snapshot上传到s3

import sagemaker

sagemaker_session = sagemaker.Session()
sagemaker_session_bucket = sagemaker_session.default_bucket()
s3_location = f"s3://{sagemaker_session_bucket}/{model_name.split('/')[-1]}/"

#你也可以把local_model_path直接替换成你的模型路径，例"model_snapshot_path=./chatglm3-6b", 这个文件夹里需要包含config.json
model_snapshot_path = local_model_path

print(f"model_snapshot_path: {model_snapshot_path}")
print("s3_location:",s3_location)

In [ ]:
#上传模型
!aws s3 sync $model_snapshot_path $s3_location

In [ ]:
#将模型的s3路径更新到inference.py中
!sed -i 's|s3_location =.*|s3_location = \"{s3_location}\"|' code/inference.py

In [ ]:
#开始部署

import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)
print(region_name)


if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_chinese')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_chinese')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'

url = f'763104351884.dkr.ecr.{region_name}.amazonaws.com/huggingface-pytorch-inference:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04'
if "cn-" in region_name:
    url = f'727897471807.dkr.ecr.{region_name}.amazonaws.com.cn/huggingface-pytorch-inference:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04'
    
from sagemaker.huggingface.model import HuggingFaceModel
model = HuggingFaceModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    image_uri=url
)


endpoint_name = 'pytorch-inference-llm-v1'
instance_type='ml.g4dn.4xlarge' 

instance_count = 1


import boto3

client = boto3.client('sagemaker')
try:
    response = client.delete_endpoint_config(EndpointConfigName=endpoint_name)
except:
    pass


from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

### 测试


In [ ]:
#休眠5分钟,确保模型可以完全加载
import time
time.sleep(300)

In [ ]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
predictor.delete_endpoint()